In [94]:
import yfinance as yF
from yahoo_fin import stock_info as si
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
import pandas_datareader.data as web
import datetime
import warnings
warnings.filterwarnings("ignore")

In [95]:
NDX_url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

In [96]:
nasdaq100 = pd.read_html(NDX_url, header=0)[4]

In [97]:
nasdaq100 = nasdaq100['Ticker']

In [98]:
ticker_list = nasdaq100.to_list()

In [99]:
#------

In [100]:
#Start and End date need to be at least one month greater than a year apart. 
start ="2022-01-01"
end = "2023-06-01"

In [101]:
#List of tickers used for the strategy.
ticker_s = ticker_list

In [102]:
#------

In [103]:
def SMA_Above(ticker_symbol):
    for column in ticker_symbol:
        ticker_symbol[column+'_SMA'] = ticker_symbol[column].rolling(168).mean()
        if ticker_symbol.iloc[-1][column+'_SMA'] > ticker_symbol.iloc[-1][column]:
            ticker_symbol.drop(column, axis=1, inplace=True)
    
    regex_str = '_SMA'
    ticker_symbol = ticker_symbol.drop(ticker_symbol.columns[ticker_symbol.columns.str.contains(regex_str)], axis=1)
    global column_names
    column_names = ticker_symbol.columns.values.tolist()

In [104]:
def get_momentum_multiple(current):
    global momentum_matrix
    #Major issues with the monthly outcomes not being accurate. 
    #------
    #1 Month
    month_1 = current.tail(1).copy()
    #3 Month
    month_3 = current.tail(3).copy()
    #6 Month
    month_6 = current.tail(6).copy()
    #9 Month
    month_9 = current.tail(9).copy()
    #12 Month
    month_12 = current.tail(12).copy()
    #Month 1 Perf.
    month_1_per = month_1
    month_1 = (month_1_per*100).round(2)
    #Month 3,6,9,12 Perf.
    df_list = month_3, month_6, month_9, month_12
    for dataframe in df_list:
        for column in dataframe:
            dataframe[column] = ((1 + dataframe[column]).cumprod()-1)
            dataframe[column] = (dataframe[column]*100).round(2)    
    #------
    #Average
    momentum_matrix = pd.concat([month_1.tail(1), month_3.tail(1), month_6.tail(1), month_9.tail(1), month_12.tail(1)], axis=0)
    momentum_matrix = momentum_matrix.mean(axis=0).round(2)
    #Removes Assets with 0 or negative momentum
    momentum_matrix = momentum_matrix.drop(momentum_matrix[momentum_matrix <= 0].index)
    #Only selects the top 5 assets
    momentum_matrix = momentum_matrix.nlargest(n=5, keep='first')

In [105]:
#------

In [106]:
#SMA Data
#Removes assets that are below thier 168 day or 8 month SMA. 
df = yF.download(ticker_s, start=start, end=end)['Adj Close']
SMA_Above(df)

[*********************100%***********************]  101 of 101 completed


In [108]:
#------

In [109]:
#Momentum
#Selects the top 5 assets based on momentum and removes assets with momentum <= 0. 
ticker_s = column_names
df1 = yF.download(ticker_s, start=start, end=end)['Adj Close']
df1 = (df1.pct_change() +1)[1:].resample('M').prod()
df1 = df1 - 1
get_momentum_multiple(df1)

[*********************100%***********************]  67 of 67 completed


In [110]:
momentum_matrix

NVDA    61.33
META    45.64
NFLX    38.44
CPRT    34.20
MELI    34.18
dtype: float64

In [111]:
path = 'C:/Users/monic/Documents/Portfolio Data/Strategy Asset Data'
output_file = os.path.join(path,'NDX_Assets.csv')
momentum_matrix.to_csv(output_file, index=True)